# PPO
PPO (Proximal Policy Optimization) ist im Kern eine vereinfachte, robuste Weiterentwicklung von TRPO – mit fast derselben Idee, aber ohne komplizierte Optimierung. Genau deshalb ist PPO heute der De‑facto‑Standard im Policy‑Gradient‑RL.

## Grundidee von PPO
PPO will Policy‑Updates stabil halten, indem es verhindert, dass die neue Policy zu weit von der alten abweicht – aber ohne die schweren mathematischen Constraints von TRPO.
Der Trick:
PPO begrenzt das Policy‑Update direkt im Loss‑Term durch Clipping.

Damit wird jeder Update-Schritt automatisch „verträglich“.

## Der Kern: Clipped Surrogate Objective
PPO maximiert eine modifizierte Version des Policy‑Gradient‑Ziels:
$$L(\theta )=\mathbb{E}\left[ \min \left( r(\theta )A,\; \mathrm{clip}(r(\theta ),1-\epsilon ,1+\epsilon )A\right) \right] $$
mit
$$r(\theta )=\frac{\pi _{\theta }(a|s)}{\pi _{\theta _{\mathrm{old}}}(a|s)}$$
Was bedeutet das?
r$$(\theta )$$ 
misst, wie stark sich die neue Policy von der alten unterscheidet.
- Wenn 
$$r(\theta )$$
 zu groß oder zu klein wird, wird es geclippt.
- Dadurch kann der Update nicht explodieren.
Ergebnis:
Die Policy verbessert sich, aber nur in einem „sicheren“ Bereich.

## Warum ist das besser als TRPO?
TRPO braucht:
- KL‑Constraint
- Conjugate Gradient
- Line Search
PPO ersetzt das alles durch einfaches Clipping im Loss.
Das ist:
- schneller
- stabiler
- GPU‑freundlich
- leichter zu implementieren
Und liefert trotzdem fast dieselbe Stabilität wie TRPO.

🔧 PPO‑Ablauf (vereinfacht)
- Rollouts sammeln
– mehrere Schritte in der Umgebung ausführen
– Zustände, Aktionen, Rewards speichern
- Advantage schätzen
– oft mit GAE (Generalized Advantage Estimation)
- Policy‑Update mit Clipped Loss
– mehrere Epochs über dieselben Daten
– Mini‑Batch‑SGD
- Value‑Function separat optimieren
– eigener Loss (MSE)
- Optional: KL‑Monitoring
– PPO bricht ab, wenn KL zu groß wird

## Warum PPO so beliebt ist
- extrem stabil
- sehr sample‑effizient
- funktioniert in kontinuierlichen Aktionsräumen
- einfach zu implementieren
- robust gegenüber Hyperparametern
- funktioniert in fast allen RL‑Benchmarks
Deshalb nutzen es viele Frameworks (SB3, RLlib, CleanRL) als Standard.

## Kurzfassung
PPO funktioniert so:
- Nutzt denselben Vorteil wie TRPO: begrenzte Policy‑Änderung
- Aber statt komplizierter Constraints nutzt es Clipping im Loss
- Dadurch wird jeder Update automatisch stabil
- Sehr effizient und leicht zu trainieren

# TRPO
TRPO („Trust Region Policy Optimization“) ist eine der elegantesten Policy‑Gradient‑Methoden, weil sie ein zentrales Problem klassischer PG‑Verfahren löst: zu große, instabile Policy‑Updates. Die Idee ist, die Policy nur so weit zu verändern, wie man ihr „vertrauen“ kann – daher Trust Region.

TRPO maximiert den erwarteten Vorteil (Advantage), unter der Nebenbedingung, dass sich die neue Policy nicht zu stark von der alten unterscheidet.
Formal:
$$\max _{\theta }\; \mathbb{E_{\mathnormal{\pi _{\theta _{\mathrm{old}}}}}}\left[ \frac{\pi _{\theta }(a|s)}{\pi _{\theta _{\mathrm{old}}}(a|s)}A(s,a)\right] $$
unter der Nebenbedingung:
$$D_{\mathrm{KL}}(\pi _{\theta _{\mathrm{old}}}\, \| \, \pi _{\theta })\leq \delta $$
Das heißt:
- Ziel: verbessere die Policy proportional zum Advantage
- Constraint: KL‑Divergenz darf nur klein sein (kleiner als δ)
Damit verhindert TRPO „Policy‑Explosionen“.

## Warum braucht man so etwas?
Normale Policy‑Gradients machen:
- einen Schritt in Richtung des Gradienten
- mit einer festen Lernrate
Das Problem:
Ein zu großer Schritt kann die Policy komplett ruinieren → Training kollabiert.
TRPO löst das, indem es mathematisch garantiert, dass jeder Schritt die Policy nur kontrolliert verändert.

## Wie TRPO das technisch löst
TRPO nutzt drei zentrale Bausteine:

1. Surrogate Objective
Statt den echten Return zu maximieren, verwendet TRPO eine „Ersatzfunktion“:
$$L(\theta )=\mathbb{E}\left[ r(\theta )A\right] $$
mit dem Wahrscheinlichkeitsverhältnis:
$$r(\theta )=\frac{\pi _{\theta }(a|s)}{\pi _{\theta _{\mathrm{old}}}(a|s)}$$
Das ist derselbe Kern wie später bei PPO.

2. Trust‑Region‑Constraint (KL‑Bound)
Die Policy darf sich nur wenig ändern:
$$D_{\mathrm{KL}}(\pi _{\mathrm{old}}\, \| \, \pi _{\theta })\leq \delta $$
Das ist ein harte Nebenbedingung (Constraint Optimization).

3. Conjugate Gradient + Line Search
Weil neuronale Netze viele Parameter haben, kann man die Optimierung nicht direkt lösen.
TRPO nutzt:
- Conjugate Gradient, um die natürliche Gradientenrichtung zu approximieren
- Backtracking Line Search, um sicherzustellen, dass der KL‑Constraint eingehalten wird
Das Ergebnis ist ein sicherer, monotonic improving Policy‑Update.

## Warum ist TRPO wichtig?
TRPO war der erste große Durchbruch, der gezeigt hat:
Man kann Policy‑Gradients stabilisieren, indem man die Policy‑Änderung begrenzt.

Das führte direkt zu:
- PPO (vereinfachte Version, heute Standard)
- Soft Actor‑Critic (ähnliche Stabilitätsideen)
- Natural Policy Gradient‑Varianten

## Kurzfassung
TRPO funktioniert so:
- Berechne Advantage‑Schätzungen
- Formuliere ein Surrogate Objective
- Finde einen Policy‑Update, der dieses Objective verbessert
- Aber: Erlaube nur kleine KL‑Änderungen
- Nutze Conjugate Gradient + Line Search, um den Constraint einzuhalten
Ergebnis: stabile, monotone Policy‑Verbesserung.
